In [1]:
from datetime import datetime
from datetime import timedelta
import json
import re
from os import path, environ
from time import sleep
import os

import requests
#from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver      # imports
from selenium.webdriver.support.ui import Select

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
base_url = 'https://www.gunviolencearchive.org'

In [3]:
#These files need to be in the same directory as the jupyter notebook file.
SHOOTINGS_JSON = 'shootings.json'

SHOOTINGS_COMPACT_JSON = 'shootings_compact.json'

### Function declarations:

In [4]:

def scrape_results(current_url):
    headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5"
}
    url = current_url
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, features="lxml")
    table = soup.find("table")
    headers = [
        re.sub(r"[^a-z]", "", th.get_text().lower()) for th in table.find_all("th")
    ]
    rows = table.find("tbody").find_all("tr")

    results = []
    for row in rows:
        cols = row.find_all("td")
        links = row.select(".links a")
        incident_id = int(re.sub(r"[^\d]", "", links[0].get("href")))
        if not incident_id > 0:
            raise RuntimeError(f"Invalid incident ID {incident_id} at {url}")
        results.append(
            {
                "id": int(re.sub(r"[^\d]", "", links[0].get("href"))),
                "date": parse(cols[headers.index("incidentdate")].get_text()),
                "state": cols[headers.index("state")].get_text(),
                "city": cols[headers.index("cityorcounty")].get_text(),
                "address": cols[headers.index("address")].get_text(),
                "killed": int(cols[headers.index("killed")].get_text()),
                "injured": int(cols[headers.index("injured")].get_text()),
                "source": links[1].get("href") if len(links) > 1 else None,
            }
        )
        
    return results

In [5]:
def merge_shooting_data(scraped_data):
    total_new = 0
    for shooting in scraped_data:
        if shooting["id"] not in keyed_shooting_data:
            print(f'New shooting found: {shooting["id"]}, {shooting["date"]}')
            keyed_shooting_data[shooting["id"]] = {}
            total_new += 1
        keyed_shooting_data[shooting["id"]].update(shooting)
    return total_new

In [6]:

def parse_page(next_url):
    # HTTP GET requests
    page = requests.get(next_url)

    # Checking if we successfully fetched the URL
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'lxml')
        try:
            current_url = next_url
            print(current_url)
            results = scrape_results(current_url)
            merge_shooting_data(results)
            next_page_text = bs.find('ul', class_="pager").findAll('li')[-2].text
            if next_page_text == 'next ›':
                next_page_partial = bs.find('ul', class_="pager").findAll('li')[-2].find('a')['href']
                next_page_url = base_url + next_page_partial
                current_url = next_page_url
                print(next_page_url)
                results = scrape_results(current_url)
                merge_shooting_data(results)
                sleep(3)
                if next_page_text == 'next ›':
                    parse_page(next_page_url)
                    sleep(1)
                else:
                    merge_shooting_data(results)
        except:
            current_url = next_url
            results = scrape_results(current_url)
            merge_shooting_data(results)
            

### Main implementation:

In [14]:
# Test
year = '2019'
month= 'Jan'
day = 1


driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

driver.get('https://www.gunviolencearchive.org/query')



add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar2.click()
sleep(4)


# Choose Date

driver.find_element_by_link_text('Date').click()

# 1 seconds for page loading
sleep(1)

# From DATE
# pull out calender
from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
from_date.click()
# month select
month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month1.select_by_visible_text(month)
sleep(1)


# year select
year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year1.select_by_visible_text(str(year))

sleep(1)


# start date

start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
start_date.click()

# TO DATE
# pull out calender
# end date
end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
end_date.click()

# month select
sleep(1)
month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
sleep(1)
month2.select_by_visible_text(month)
sleep(1)


# year select
year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
sleep(1)
year2.select_by_visible_text(str(year))

sleep(1)

# End date
# Currently, I made up first available date, which 0 index from the list.
End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day]
End_date.click()

sleep(1)

# Click add a rule
add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
add_rule_bar.click()
sleep(3)

# Select 'Incident Characteristic'

driver.find_element_by_link_text('Incident Characteristic').click()

# select Accidental Shooting

acc_shoot = driver.find_elements_by_xpath("/html/body/section/div/div/div/div[2]/div/form/div/div[3]/div[2]/div[2]/div[2]/div[2]/div/span/select/option")

acc_shoot.click()
# Click search
driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()

IndexError: list index out of range

In [7]:
year = ['2019']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        else:
            if month[months]=='Jan' or month[months]=='Mar' or month[months]=='May' or month[months]=='Jul' or month[months]=='Aug' or month[months]=='Oct' or month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            elif month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27] 
            else :
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')
                
                
                
                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)
                
                
                # Choose Date
                
                driver.find_element_by_link_text('Date').click()
                
                # 1 seconds for page loading
                sleep(1)
                
                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month1.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year1.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                
                # start date
                
                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()
                
                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()
                
                # month select
                sleep(1)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(1)
                month2.select_by_visible_text(month[months])
                sleep(1)
                
                
                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(1)
                year2.select_by_visible_text(str(year[years]))
                
                sleep(1)
                
                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()
                
                sleep(1)
                
                # Click add a rule
                add_rule_bar = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar.click()
                sleep(1)
                
                # Select 'Incident Characteristic'
                
                driver.find_element_by_link_text('Incident Characteristic').click()
                
                # Click search
                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                parse_page(current_url)
                print("Sorting results")
                updated_shooting_data = sorted(
                    keyed_shooting_data.values(), key=lambda row: row['date']
                )

                updated_shootings_compact = []
                for shooting in updated_shooting_data:
                    shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                    updated_shootings_compact.append(
                        [
                            shooting["id"],
                            shooting["date"],
                            shooting["killed"],
                            shooting["injured"],

                        ]
                    )


                print("Writing results")
                with open(SHOOTINGS_JSON, "w") as outfile:
                    json.dump(updated_shooting_data, outfile, indent=2)

                with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                    json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

                print(f"Done! {len(updated_shooting_data)} shootings found")
                driver.close()
            else:
                break
            


Loading existing shooting data with 263 incidents
https://www.gunviolencearchive.org/query/6234df41-6085-4d80-a546-36499e5f8f15
New shooting found: 1292931, 2019-01-01 00:00:00
New shooting found: 1290089, 2019-01-01 00:00:00
New shooting found: 1289952, 2019-01-01 00:00:00
New shooting found: 1292119, 2019-01-01 00:00:00
New shooting found: 1290429, 2019-01-01 00:00:00
New shooting found: 1290421, 2019-01-01 00:00:00
New shooting found: 1289910, 2019-01-01 00:00:00
New shooting found: 1289680, 2019-01-01 00:00:00
New shooting found: 1289462, 2019-01-01 00:00:00
New shooting found: 1291901, 2019-01-01 00:00:00
New shooting found: 1291237, 2019-01-01 00:00:00
New shooting found: 1291205, 2019-01-01 00:00:00
New shooting found: 1290007, 2019-01-01 00:00:00
New shooting found: 1289595, 2019-01-01 00:00:00
New shooting found: 1290356, 2019-01-01 00:00:00
New shooting found: 1289551, 2019-01-01 00:00:00
New shooting found: 1587035, 2019-01-01 00:00:00
New shooting found: 1290847, 2019-01-01

https://www.gunviolencearchive.org/query/6234df41-6085-4d80-a546-36499e5f8f15?page=5
https://www.gunviolencearchive.org/query/6234df41-6085-4d80-a546-36499e5f8f15?page=6
New shooting found: 1289918, 2019-01-01 00:00:00
New shooting found: 1289842, 2019-01-01 00:00:00
New shooting found: 1292358, 2019-01-01 00:00:00
New shooting found: 1289901, 2019-01-01 00:00:00
New shooting found: 1290779, 2019-01-01 00:00:00
New shooting found: 1289852, 2019-01-01 00:00:00
New shooting found: 1290773, 2019-01-01 00:00:00
New shooting found: 1290552, 2019-01-01 00:00:00
New shooting found: 1292229, 2019-01-01 00:00:00
New shooting found: 1290765, 2019-01-01 00:00:00
New shooting found: 1296945, 2019-01-01 00:00:00
New shooting found: 1290152, 2019-01-01 00:00:00
New shooting found: 1289889, 2019-01-01 00:00:00
New shooting found: 1290760, 2019-01-01 00:00:00
New shooting found: 1291109, 2019-01-01 00:00:00
New shooting found: 1290755, 2019-01-01 00:00:00
New shooting found: 1289869, 2019-01-01 00:00:

New shooting found: 1291133, 2019-01-02 00:00:00
New shooting found: 1292601, 2019-01-02 00:00:00
New shooting found: 1291775, 2019-01-02 00:00:00
New shooting found: 1291786, 2019-01-02 00:00:00
New shooting found: 1292754, 2019-01-02 00:00:00
New shooting found: 1291063, 2019-01-02 00:00:00
New shooting found: 1293372, 2019-01-02 00:00:00
New shooting found: 1290549, 2019-01-02 00:00:00
New shooting found: 1290993, 2019-01-02 00:00:00
New shooting found: 1292085, 2019-01-02 00:00:00
New shooting found: 1291112, 2019-01-02 00:00:00
New shooting found: 1292714, 2019-01-02 00:00:00
New shooting found: 1291102, 2019-01-02 00:00:00
New shooting found: 1291732, 2019-01-02 00:00:00
New shooting found: 1291174, 2019-01-02 00:00:00
New shooting found: 1295954, 2019-01-02 00:00:00
New shooting found: 1291726, 2019-01-02 00:00:00
New shooting found: 1291714, 2019-01-02 00:00:00
New shooting found: 1291069, 2019-01-02 00:00:00
New shooting found: 1290568, 2019-01-02 00:00:00
New shooting found: 

https://www.gunviolencearchive.org/query/2ae302da-0884-4975-b211-01c7d32e8a1a?page=1
https://www.gunviolencearchive.org/query/2ae302da-0884-4975-b211-01c7d32e8a1a?page=2
New shooting found: 1291578, 2019-01-03 00:00:00
New shooting found: 1427920, 2019-01-03 00:00:00
New shooting found: 1299156, 2019-01-03 00:00:00
New shooting found: 1292646, 2019-01-03 00:00:00
New shooting found: 1291740, 2019-01-03 00:00:00
New shooting found: 1292638, 2019-01-03 00:00:00
New shooting found: 1292617, 2019-01-03 00:00:00
New shooting found: 1291721, 2019-01-03 00:00:00
New shooting found: 1292610, 2019-01-03 00:00:00
New shooting found: 1292604, 2019-01-03 00:00:00
New shooting found: 1291744, 2019-01-03 00:00:00
New shooting found: 1292011, 2019-01-03 00:00:00
New shooting found: 1292632, 2019-01-03 00:00:00
New shooting found: 1291718, 2019-01-03 00:00:00
New shooting found: 1292555, 2019-01-03 00:00:00
New shooting found: 1291536, 2019-01-03 00:00:00
New shooting found: 1292280, 2019-01-03 00:00:

https://www.gunviolencearchive.org/query/3f63e1ae-daef-45fc-a33d-ec6842cdc5ec?page=1
https://www.gunviolencearchive.org/query/3f63e1ae-daef-45fc-a33d-ec6842cdc5ec?page=2
New shooting found: 1295253, 2019-01-04 00:00:00
New shooting found: 1292504, 2019-01-04 00:00:00
New shooting found: 1292540, 2019-01-04 00:00:00
New shooting found: 1292483, 2019-01-04 00:00:00
New shooting found: 1292368, 2019-01-04 00:00:00
New shooting found: 1292466, 2019-01-04 00:00:00
New shooting found: 1299096, 2019-01-04 00:00:00
New shooting found: 1297390, 2019-01-04 00:00:00
New shooting found: 1292863, 2019-01-04 00:00:00
New shooting found: 1293179, 2019-01-04 00:00:00
New shooting found: 1292489, 2019-01-04 00:00:00
New shooting found: 1297018, 2019-01-04 00:00:00
New shooting found: 1292460, 2019-01-04 00:00:00
New shooting found: 1293568, 2019-01-04 00:00:00
New shooting found: 1292556, 2019-01-04 00:00:00
New shooting found: 1292431, 2019-01-04 00:00:00
New shooting found: 1292458, 2019-01-04 00:00:

https://www.gunviolencearchive.org/query/f7dbed97-c5cc-496d-8919-106c89e6b633?page=1
https://www.gunviolencearchive.org/query/f7dbed97-c5cc-496d-8919-106c89e6b633?page=2
New shooting found: 1296783, 2019-01-05 00:00:00
New shooting found: 1294667, 2019-01-05 00:00:00
New shooting found: 1293299, 2019-01-05 00:00:00
New shooting found: 1299157, 2019-01-05 00:00:00
New shooting found: 1293285, 2019-01-05 00:00:00
New shooting found: 1294779, 2019-01-05 00:00:00
New shooting found: 1295941, 2019-01-05 00:00:00
New shooting found: 1293938, 2019-01-05 00:00:00
New shooting found: 1293258, 2019-01-05 00:00:00
New shooting found: 1293706, 2019-01-05 00:00:00
New shooting found: 1295929, 2019-01-05 00:00:00
New shooting found: 1293365, 2019-01-05 00:00:00
New shooting found: 1293707, 2019-01-05 00:00:00
New shooting found: 1294746, 2019-01-05 00:00:00
New shooting found: 1294868, 2019-01-05 00:00:00
New shooting found: 1293352, 2019-01-05 00:00:00
New shooting found: 1293753, 2019-01-05 00:00:

https://www.gunviolencearchive.org/query/a7be87bd-9cf1-452c-b258-f6f76b3ecb71?page=1
https://www.gunviolencearchive.org/query/a7be87bd-9cf1-452c-b258-f6f76b3ecb71?page=2
New shooting found: 1296525, 2019-01-06 00:00:00
New shooting found: 1294204, 2019-01-06 00:00:00
New shooting found: 1295281, 2019-01-06 00:00:00
New shooting found: 1294080, 2019-01-06 00:00:00
New shooting found: 1294413, 2019-01-06 00:00:00
New shooting found: 1294800, 2019-01-06 00:00:00
New shooting found: 1294756, 2019-01-06 00:00:00
New shooting found: 1294676, 2019-01-06 00:00:00
New shooting found: 1294301, 2019-01-06 00:00:00
New shooting found: 1293969, 2019-01-06 00:00:00
New shooting found: 1294719, 2019-01-06 00:00:00
New shooting found: 1299636, 2019-01-06 00:00:00
New shooting found: 1294290, 2019-01-06 00:00:00
New shooting found: 1297425, 2019-01-06 00:00:00
New shooting found: 1294215, 2019-01-06 00:00:00
New shooting found: 1294707, 2019-01-06 00:00:00
New shooting found: 1295303, 2019-01-06 00:00:

https://www.gunviolencearchive.org/query/6604269d-fa2c-4d57-ace7-055d72762216?page=2
https://www.gunviolencearchive.org/query/6604269d-fa2c-4d57-ace7-055d72762216?page=3
New shooting found: 1295208, 2019-01-07 00:00:00
New shooting found: 1295181, 2019-01-07 00:00:00
New shooting found: 1295750, 2019-01-07 00:00:00
New shooting found: 1295109, 2019-01-07 00:00:00
New shooting found: 1295330, 2019-01-07 00:00:00
New shooting found: 1294774, 2019-01-07 00:00:00
New shooting found: 1295289, 2019-01-07 00:00:00
New shooting found: 1297841, 2019-01-07 00:00:00
New shooting found: 1296466, 2019-01-07 00:00:00
New shooting found: 1295808, 2019-01-07 00:00:00
New shooting found: 1294899, 2019-01-07 00:00:00
New shooting found: 1295283, 2019-01-07 00:00:00
New shooting found: 1295768, 2019-01-07 00:00:00
New shooting found: 1720006, 2019-01-07 00:00:00
New shooting found: 1295766, 2019-01-07 00:00:00
New shooting found: 1295839, 2019-01-07 00:00:00
New shooting found: 1295751, 2019-01-07 00:00:

https://www.gunviolencearchive.org/query/02203bc2-e326-4ee4-9491-1605bec61c64?page=3
https://www.gunviolencearchive.org/query/02203bc2-e326-4ee4-9491-1605bec61c64?page=4
New shooting found: 1296432, 2019-01-08 00:00:00
New shooting found: 1296901, 2019-01-08 00:00:00
New shooting found: 1296392, 2019-01-08 00:00:00
New shooting found: 1297282, 2019-01-08 00:00:00
New shooting found: 1297054, 2019-01-08 00:00:00
New shooting found: 1296405, 2019-01-08 00:00:00
New shooting found: 1296950, 2019-01-08 00:00:00
New shooting found: 1416459, 2019-01-08 00:00:00
New shooting found: 1296401, 2019-01-08 00:00:00
New shooting found: 1296387, 2019-01-08 00:00:00
New shooting found: 1296681, 2019-01-08 00:00:00
New shooting found: 1296892, 2019-01-08 00:00:00
New shooting found: 1296358, 2019-01-08 00:00:00
New shooting found: 1296279, 2019-01-08 00:00:00
New shooting found: 1297231, 2019-01-08 00:00:00
New shooting found: 1295347, 2019-01-08 00:00:00
New shooting found: 1296881, 2019-01-08 00:00:

KeyboardInterrupt: 

In [10]:
# Change the year, and months based on the date range you need to scarape data for. 
# So if you want data from 12th-April-2020 to 22nd-April-2020, remove all the years prior to 2020;
# and in month remove all months except 'Apr'.
# To get an even more specific date range (to save time) you can remove the corresponding days of that month from day[] in
# the corresponding if statement for that month. 
year = ['2014','2015','2016','2017','2018','2019','2020']
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']


for years in range(len(year)):
    for months in range(len(month)):
        
        if year[years]=='2020' or year[years]=='2016':
            if month[months]=='Jan':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28]
            if month[months]=='Mar':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Apr':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29] 
            if month[months]=='May':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Jun':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Jul':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Aug':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Sep':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Oct':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Nov':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            
        else:
            if month[months]=='Jan':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Feb':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27]
            if month[months]=='Mar':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Apr':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29] 
            if month[months]=='May':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Jun':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Jul':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Aug':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Sep':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Oct':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
            if month[months]=='Nov':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]
            if month[months]=='Dec':
                day = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
        for days in range(len(day)):
            
            c_date = '{}-{}-{}'.format(year[years],month[months],day[days]+1)
            c_date = datetime.strptime(c_date,"%Y-%b-%d")
            day_difference = (datetime.now() - c_date).days
            if (day_difference >3):
            
                driver = webdriver.Chrome('/Users/jasonlim/Desktop/EFGS/GVA-master/All shootings/Scripts/chromedriver')

                # open some page using get method       - url -- > parameters

                driver.get('https://www.gunviolencearchive.org/query')



                add_rule_bar2 = driver.find_element_by_class_name('filter-dropdown-trigger')
                add_rule_bar2.click()
                sleep(4)


                # Choose Date

                driver.find_element_by_link_text('Date').click()

                # 4 seconds for page loading
                sleep(4)

                # From DATE
                # pull out calender
                from_date = driver.find_element_by_css_selector("input[id$='filter-field-date-from']")
                from_date.click()
                # month select
                month1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(4)
                month1.select_by_visible_text(month[months])
                sleep(4)



                # year select
                year1 = Select(driver.find_element_by_xpath('/html/body/div[4]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year1.select_by_visible_text(str(year[years]))

                sleep(3)

                # start date

                start_date = driver.find_elements_by_xpath("/html/body/div[4]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                start_date.click()

                # TO DATE
                # pull out calender
                # end date
                end_date = driver.find_element_by_css_selector("input[id$='filter-field-date-to']")
                end_date.click()

                # month select
                sleep(3)
                month2= Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[1]'))
                sleep(3)
                month2.select_by_visible_text(month[months])
                sleep(3)


                # year select
                year2 = Select(driver.find_element_by_xpath('/html/body/div[5]/div[1]/div/table/thead/tr[1]/th[2]/select[2]'))
                sleep(3)
                year2.select_by_visible_text(str(year[years]))

                sleep(3)

                # End date
                # Currently, I made up first available date, which 0 index from the list.
                End_date = driver.find_elements_by_xpath("/html/body/div[5]/div[1]/div/table/tbody/tr/td[@class='available']")[day[days]]
                End_date.click()

                sleep(3)
                # Click Search

                driver.find_element_by_css_selector("input[id$='edit-actions-execute']").click()
                
                
                start_year = 2014
                current_year = datetime.now().year
                with open(SHOOTINGS_JSON) as json_file:
                    shooting_data = json.load(json_file)

                keyed_shooting_data = {}
                for shooting in shooting_data:
                    keyed_shooting_data[shooting["id"]] = shooting
                    shooting["date"] = parse(shooting["date"])

                print(f"Loading existing shooting data with {len(shooting_data)} incidents")
                current_url = driver.current_url
                parse_page(current_url)
                print("Sorting results")
                updated_shooting_data = sorted(
                    keyed_shooting_data.values(), key=lambda row: row['date']
                )

                updated_shootings_compact = []
                for shooting in updated_shooting_data:
                    shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
                    updated_shootings_compact.append(
                        [
                            shooting["id"],
                            shooting["date"],
                            shooting["killed"],
                            shooting["injured"],

                        ]
                    )


                print("Writing results")
                with open(SHOOTINGS_JSON, "w") as outfile:
                    json.dump(updated_shooting_data, outfile, indent=2)

                with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
                    json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

                print(f"Done! {len(updated_shooting_data)} shootings found")
                driver.close()
            else:
                break
            


Loading existing shooting data with 3 incidents
https://www.gunviolencearchive.org/query/fe09c73a-97e9-407b-a5bb-150b19d39a92
New shooting found: 92540, 2014-01-01 00:00:00
New shooting found: 92168, 2014-01-01 00:00:00
New shooting found: 95234, 2014-01-01 00:00:00
New shooting found: 95232, 2014-01-01 00:00:00
New shooting found: 92432, 2014-01-01 00:00:00
New shooting found: 92211, 2014-01-01 00:00:00
New shooting found: 92215, 2014-01-01 00:00:00
New shooting found: 92342, 2014-01-01 00:00:00
New shooting found: 92151, 2014-01-01 00:00:00
New shooting found: 92245, 2014-01-01 00:00:00
New shooting found: 92158, 2014-01-01 00:00:00
New shooting found: 92148, 2014-01-01 00:00:00
New shooting found: 92137, 2014-01-01 00:00:00
New shooting found: 104905, 2014-01-01 00:00:00
New shooting found: 92734, 2014-01-01 00:00:00
New shooting found: 92736, 2014-01-01 00:00:00
New shooting found: 94194, 2014-01-01 00:00:00
New shooting found: 261045, 2014-01-01 00:00:00
New shooting found: 92160,

https://www.gunviolencearchive.org/query/fe09c73a-97e9-407b-a5bb-150b19d39a92?page=6
https://www.gunviolencearchive.org/query/fe09c73a-97e9-407b-a5bb-150b19d39a92?page=7
New shooting found: 92376, 2014-01-01 00:00:00
New shooting found: 92560, 2014-01-01 00:00:00
New shooting found: 92828, 2014-01-01 00:00:00
New shooting found: 95255, 2014-01-01 00:00:00
New shooting found: 150401, 2014-01-01 00:00:00
https://www.gunviolencearchive.org/query/fe09c73a-97e9-407b-a5bb-150b19d39a92?page=7
Sorting results
Writing results
Done! 183 shootings found
Loading existing shooting data with 183 incidents
https://www.gunviolencearchive.org/query/1e5416f7-e975-4ba2-b5de-6b5672732417
New shooting found: 92366, 2014-01-02 00:00:00
New shooting found: 152090, 2014-01-02 00:00:00
New shooting found: 213872, 2014-01-02 00:00:00
New shooting found: 92549, 2014-01-02 00:00:00
New shooting found: 92508, 2014-01-02 00:00:00
New shooting found: 92277, 2014-01-02 00:00:00
New shooting found: 92299, 2014-01-02 0

KeyboardInterrupt: 

## Test code 

In [29]:
# # TEST LOOP:

# year = ["2019","2020"]
# month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

# count=0
# for years in range(len(year)): 
    
#     for months in range(len(month)):
        
                
#         print(day)
        
           
            



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 2

In [ ]:
# Opening and reading files test code:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

In [15]:
# start_year = 2014
# current_year = datetime.now().year
# with open(SHOOTINGS_JSON) as json_file:
#     shooting_data = json.load(json_file)

# keyed_shooting_data = {}
# for shooting in shooting_data:
#     keyed_shooting_data[shooting["id"]] = shooting
#     shooting["date"] = parse(shooting["date"])

# print(f"Loading existing shooting data with {len(shooting_data)} incidents")

# # Insert any GVA date query url link into the parse page function to test the code
# parse_page('https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f')
# print("Sorting results")
# updated_shooting_data = sorted(
#     keyed_shooting_data.values(), key=lambda row: row['date']
# )

# updated_shootings_compact = []
# for shooting in updated_shooting_data:
#     shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#     updated_shootings_compact.append(
#         [
#             shooting["id"],
#             shooting["date"],
#             shooting["killed"],
#             shooting["injured"],

#         ]
#     )


# print("Writing results")
# with open(SHOOTINGS_JSON, "w") as outfile:
#     json.dump(updated_shooting_data, outfile, indent=2)

# with open(SHOOTINGS_COMPACT_JSON, "w") as outfile:
#     json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

# print(f"Done! {len(updated_shooting_data)} shootings found")

Loading existing shooting data with 56112 incidents
https://www.gunviolencearchive.org/query/b6bf79c7-0651-4713-8eca-cae8e66e150f
Sorting results
Writing results
Done! 56112 shootings found


In [38]:
# y = ['2020']
# m = ['Apr']
# d = [10]
# for ys in range(len(y)):
#     for ms in range(len(m)):
#         for ds in range(len(d)):
#             c_date = '{}-{}-{}'.format(y[ys],m[ms],d[ds])
#             c_date = datetime.strptime(c_date,"%Y-%b-%d")
#             day_difference = (datetime.now() - c_date).days
#             print(day_difference)
#             if (day_difference >3):
#                 print(c_date)
#             else:
#                 print("cats")

#### Geocoder test code

In [ ]:
# for year in range(start_year, current_year + 1):
#     print(f"scraping year {year}")
#     page = 0
#     done_importing = False
#     while not done_importing:
#         results = scrape_results(year, page, headers)
#         new_results_count = merge_shooting_data(results)
#         done_importing = new_results_count == 0
#         page += 1
#         sleep(1)
#     print(f"Done importing {year}")

# # print("Geocoding missing data")
# # for shooting in keyed_mass_shooting_data.values():
# #     if not shooting.get("lat") or not shooting.get("lng"):
# #         loc = geocode(shooting["address"], shooting["city"], shooting["state"])
# #         shooting["lat"] = loc.latitude
# #         shooting["lng"] = loc.longitude
# #         print(f"geocoded {loc.address}: {loc.latitude}, {loc.longitude}")

#### Update results method test (did not use in implementation)

In [ ]:
# def update_results():

#     start_year = 2014
#     current_year = datetime.now().year
#     with open(ACCIDENTAL_SHOOTINGS_JSON) as json_file:
#         accidental_shooting_data = json.load(json_file)

#     keyed_accidental_shooting_data = {}
#     for shooting in accidental_shooting_data:
#         keyed_accidental_shooting_data[shooting["id"]] = shooting
#         shooting["date"] = parse(shooting["date"])

#     print(f"Loading existing accidental shooting data with {len(accidental_shooting_data)} incidents")
#     print("Sorting results")
#     updated_shooting_data = sorted(
#         keyed_accidental_shooting_data.values(), key=lambda row: row['date']
#     )

#     updated_shootings_compact = []
#     for shooting in updated_shooting_data:
#         shooting["date"] = shooting["date"].strftime("%Y-%m-%d")
#         updated_shootings_compact.append(
#             [
#                 shooting["id"],
#                 shooting["date"],
#                 shooting["killed"],
#                 shooting["injured"],

#             ]
#         )


#     print("Writing results")
#     with open(ACCIDENTAL_SHOOTINGS_JSON, "w") as outfile:
#         json.dump(updated_shooting_data, outfile, indent=2)

#     with open(ACCIDENTAL_SHOOTINGS_COMPACT_JSON, "w") as outfile:
#         json.dump(updated_shootings_compact, outfile, separators=(",", ":"))

#     print(f"Done! {len(updated_shooting_data)} shootings found")